<a href="https://colab.research.google.com/github/alengeorge-26/Plant-Disease-Classifier/blob/main/Plant_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE =256
BATCH_SIZE=32
DIRECTORY="/content/drive/MyDrive/Plant_DIsease/PlantVillage"

In [ ]:
dataset=tf.keras.utils.image_dataset_from_directory(
  DIRECTORY,
   shuffle=True,
   image_size=(IMAGE_SIZE,IMAGE_SIZE),
   batch_size=BATCH_SIZE
)

In [ ]:
class_names=dataset.class_names
class_names

In [ ]:
train_size=.8
len(dataset)*train_size

In [ ]:
train_ds=dataset.take(54)
testing_ds=dataset.skip(54)
len(testing_ds)

14

In [ ]:
val_ds=testing_ds.take(6)
test_ds=testing_ds.skip(6)
len(test_ds)

8